In [60]:
%cd /content/gdrive/MyDrive/"Colab Notebooks"/llm_study
!ls
!git init
!git add llm_study.ipynb
!git commit -m "first commit"
!git remote add origin https://github.com/minseok-blog/llm_study.git
!git push -u origin main

/content/gdrive/MyDrive/Colab Notebooks/llm_study
llm_study.ipynb
Reinitialized existing Git repository in /content/gdrive/MyDrive/Colab Notebooks/llm_study/.git/
[master (root-commit) 410cc04] first commit
 1 file changed, 1 insertion(+)
 create mode 100644 llm_study.ipynb
error: src refspec main does not match any
error: failed to push some refs to 'origin'


# HuggingFace에서 모델을 설치받고, 입력값에 대한 모델의 출력값 확인하기

In [1]:
!pip install transformers
!pip install huggingface-hub
!pip install --upgrade transformers
!huggingface-cli login

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')
model = AutoModelForCausalLM.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')
print(model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [6]:
text = "What is the capital of France?"
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
outputs = model.generate(**inputs)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

# CSV Dataset을 이용해서 파인튜닝하기 - 데이터셋 불러오기, HF에 데이터셋 업로드하기


In [4]:
!pip install jsonlines
!pip install datasets

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [9]:
import json
import jsonlines
from datasets import Dataset
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
filename = "indata_kor"
dataset_Path = "/content/gdrive/MyDrive/Colab Notebooks"
csv_file_path = os.path.join(dataset_Path, f"{filename}.csv")

In [11]:
try :
  df = pd.read_csv(csv_file_path, encoding='ms949')
except Exception as e :
  df = pd.read_csv(csv_file_path, encoding='utf-8')

In [12]:
df["input"] = ""
df.columns = ["instruction", "output", "input"]
df.head(2)

,instruction,output,input
0,유튜브 채널 hkcode에서는 무엇을 가르치나요?,"초보자 대상으로 빅데이터, 인공지능과 관련된 컨텐츠를 가르치고 있습니다.",
1,유튜브 채널 hkcode는 누가 운영하나요?,한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다.,


In [9]:
dataset = Dataset.from_pandas(df)

In [10]:
dataset.push_to_hub("MinseokBlog/llama3data-hkcode")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/MinseokBlog/llama3data-hkcode/commit/4a4509dcc7d11701c3b663c858711af205a8961d', commit_message='Upload dataset', commit_description='', oid='4a4509dcc7d11701c3b663c858711af205a8961d', pr_url=None, pr_revision=None, pr_num=None)

# CSV Dataset을 이용하여 파인튜닝하기2 -


In [11]:
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.40.1 trl==0.8.6 datasets==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 11.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_

In [1]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import huggingface_hub
huggingface_hub.login()


In [4]:
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
new_model = "Llama3-minseok-Ko-8b"

In [7]:
dataset_name = "hyokwan/localllama"
datasethk = load_dataset(dataset_name, split="train")

Generating train split:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
def create_text_column(example):
  text = f"### Instruction: {example['instruction']}\n### Input: {example['input']}\n### Response: {example['output']}"
  example["text"] = text
  return example

datasethk = datasethk.map(create_text_column)